# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [14]:
%env CUDA_VISIBLE_DEVICES=3,4
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=3,4


In [15]:
window_size = 5001
sample_num = 10000

In [16]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [17]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [18]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [19]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [20]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [21]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 5001
[[123409527, 123414528], [123409441, 123414442], [131906064, 131911065]]


In [22]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 5001, 5)


In [23]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [24]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 5001, 5)


In [25]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 5001)


In [26]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 5001)


In [27]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 5001, 1)


In [28]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.97687  0.97687  0.97687  0.97687  0.97687]


In [29]:
from keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [30]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 300
evaluation_freq = 10

In [31]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1,
    name='gen_conv1d_1')(inputs)
x = Dropout(dropout_rate,
            name='gen_dropout_1')(x)
# x = BatchNormalization(momentum=0.8)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1,
    name='gen_conv1d_output'
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [32]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5001, 5)           0         
_________________________________________________________________
gen_conv1d_1 (Conv1D)        (None, 5001, 32)          800192    
_________________________________________________________________
gen_dropout_1 (Dropout)      (None, 5001, 32)          0         
_________________________________________________________________
gen_conv1d_output (Conv1D)   (None, 5001, 1)           1025      
Total params: 801,217
Trainable params: 801,217
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [34]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [35]:
model_dir = os.path.join("models", "cnn_5000_new")
log_dir = os.path.join("logs", "cnn_5000_new")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "cnn_5000_new")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [36]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=1e-3, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [37]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [38]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [39]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [43]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(int(sample_num*0.8), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:int(sample_num*0.8)], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(int(sample_num*0.2), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[int(sample_num*0.8):sample_num], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(int(sample_num*0.1), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[sample_num:int(sample_num*1.1)], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [44]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [45]:
print "Fitting the model..."
train_index = int(sample_num*0.8)
val_index = sample_num
test_index = int(sample_num*1.1) 
X_train, y_train = atac_seq_day0[:train_index], histone_mark_day0[:train_index]
X_val, y_val = atac_seq_day0[train_index:val_index], histone_mark_day0[train_index:val_index]
X_test, y_test = atac_seq_day0[val_index:test_index], histone_mark_day0[val_index:test_index]

model.fit(atac_seq_day0[:sample_num],
          histone_mark_day0[:sample_num],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.2,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 8000 samples, validate on 2000 samples
Epoch 1/300
7936/8000 [============================>.] - ETA: 0s - loss: 224.1588 - pearson: 0.1947 - mean_squared_error: 224.1588 - mean_absolute_error: 5.4322
Epoch 00001: val_loss improved from inf to 77.34403, saving model to models/cnn_5000_new/best_model.h5
Writing 1D track of shape: (8000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
8000/8000 [==============================] - 461s 58ms/step - loss: 225.2602 - pearson: 0.1955 - mean_squared_error: 225.2602 - mean_absolute_error: 5.4444 - val_loss: 77.3440 - val_pearson: 0.1544 - va

Test spearman: 0.303189287088

Epoch 00010: val_loss improved from 73.17261 to 72.62055, saving model to models/cnn_5000_new/best_model.h5
Writing 1D track of shape: (8000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
8000/8000 [==============================] - 432s 54ms/step - loss: 214.8422 - pearson: 0.2281 - mean_squared_error: 214.8422 - mean_absolute_error: 5.2425 - val_loss: 72.6206 - val_pearson: 0.1738 - val_mean_squared_error: 72.6206 - val_mean_absolute_error: 2.6822
Epoch 11/300
7936/8000 [============================>.] - ETA: 0s - loss: 215.0160 - pearson: 0.2283 - mean_squared_error: 215.016

7936/8000 [============================>.] - ETA: 0s - loss: 208.7807 - pearson: 0.2477 - mean_squared_error: 208.7807 - mean_absolute_error: 5.3146
Epoch 00023: val_loss improved from 71.42208 to 71.31808, saving model to models/cnn_5000_new/best_model.h5
Writing 1D track of shape: (8000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
8000/8000 [==============================] - 451s 56ms/step - loss: 209.9978 - pearson: 0.2476 - mean_squared_error: 209.9978 - mean_absolute_error: 5.3195 - val_loss: 71.3181 - val_pearson: 0.1854 - val_mean_squared_error: 71.3181 - val_mean_absolute_error: 2.5651
Epoch 24/300

Epoch 37/300
7936/8000 [============================>.] - ETA: 0s - loss: 207.7605 - pearson: 0.2558 - mean_squared_error: 207.7605 - mean_absolute_error: 5.3231
Epoch 00037: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 206.7130 - pearson: 0.2551 - mean_squared_error: 206.7130 - mean_absolute_error: 5.3094 - val_loss: 71.4208 - val_pearson: 0.1822 - val_mean_squared_error: 71.4208 - val_mean_absolute_error: 2.4472
Epoch 38/300
7936/8000 [============================>.] - ETA: 0s - loss: 209.0041 - pearson: 0.2526 - mean_squared_error: 209.0041 - mean_absolute_error: 5.3036
Epoch 00038: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 208.3121 - pearson: 0.2521 - mean_squared_error: 208.3121 - mean_absolute_error: 5.3035 - val_loss: 71.7941 - val_pearson: 0.1800 - val_mean_squared_error: 71.7941 - val_mean_absolute_error: 2.7553
Epoch 39/300
7936/8000 [============================>.] - ETA: 0s - lo

Epoch 53/300
7936/8000 [============================>.] - ETA: 0s - loss: 206.1565 - pearson: 0.2624 - mean_squared_error: 206.1565 - mean_absolute_error: 5.2837
Epoch 00053: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 204.9579 - pearson: 0.2619 - mean_squared_error: 204.9579 - mean_absolute_error: 5.2704 - val_loss: 72.2405 - val_pearson: 0.1788 - val_mean_squared_error: 72.2405 - val_mean_absolute_error: 2.2771
Epoch 54/300
7936/8000 [============================>.] - ETA: 0s - loss: 204.4074 - pearson: 0.2616 - mean_squared_error: 204.4074 - mean_absolute_error: 5.2200
Epoch 00054: val_loss improved from 70.65409 to 70.60575, saving model to models/cnn_5000_new/best_model.h5
Writing 1D track of shape: (8000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000_new/val.bw
Wrote 

...

1000/1000 [==============================] - 4s 4ms/steposs: 197.1275 - pearson: 0.287
[('loss', 115.11474047851563), ('pearson', 0.27525403952598571), ('mean_squared_error', 115.11474102783203), ('mean_absolute_error', 3.6159437236785887)]
Test spearman: 0.30466400309

Epoch 00220: val_loss did not improve
8000/8000 [==============================] - 85s 11ms/step - loss: 196.3526 - pearson: 0.2885 - mean_squared_error: 196.3526 - mean_absolute_error: 5.2018 - val_loss: 70.3842 - val_pearson: 0.1840 - val_mean_squared_error: 70.3842 - val_mean_absolute_error: 2.5448
Epoch 221/300
7936/8000 [============================>.] - ETA: 0s - loss: 197.2586 - pearson: 0.2892 - mean_squared_error: 197.2586 - mean_absolute_error: 5.1574
Epoch 00221: val_loss did not improve
8000/8000 [==============================] - 76s 9ms/step - loss: 196.6250 - pearson: 0.2889 - mean_squared_error: 196.6250 - mean_absolute_error: 5.1548 - val_loss: 70.3731 - val_pearson: 0.1832 - val_mean_squared_error: 70.

7936/8000 [============================>.] - ETA: 0s - loss: 196.1488 - pearson: 0.2888 - mean_squared_error: 196.1488 - mean_absolute_error: 5.1214
Epoch 00237: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 196.0221 - pearson: 0.2897 - mean_squared_error: 196.0221 - mean_absolute_error: 5.1197 - val_loss: 70.5325 - val_pearson: 0.1820 - val_mean_squared_error: 70.5325 - val_mean_absolute_error: 2.5434
Epoch 238/300
7936/8000 [============================>.] - ETA: 0s - loss: 196.2540 - pearson: 0.2933 - mean_squared_error: 196.2540 - mean_absolute_error: 5.1196
Epoch 00238: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 196.5972 - pearson: 0.2934 - mean_squared_error: 196.5972 - mean_absolute_error: 5.1259 - val_loss: 70.4012 - val_pearson: 0.1838 - val_mean_squared_error: 70.4012 - val_mean_absolute_error: 2.5889
Epoch 239/300
7936/8000 [============================>.] - ETA: 0s - loss: 197.151

7936/8000 [============================>.] - ETA: 0s - loss: 196.4104 - pearson: 0.2892 - mean_squared_error: 196.4104 - mean_absolute_error: 5.1693
Epoch 00254: val_loss did not improve
8000/8000 [==============================] - 76s 10ms/step - loss: 195.8336 - pearson: 0.2894 - mean_squared_error: 195.8336 - mean_absolute_error: 5.1644 - val_loss: 70.1778 - val_pearson: 0.1869 - val_mean_squared_error: 70.1778 - val_mean_absolute_error: 2.6249
Epoch 255/300
7936/8000 [============================>.] - ETA: 0s - loss: 194.4082 - pearson: 0.2909 - mean_squared_error: 194.4082 - mean_absolute_error: 5.2021
Epoch 00255: val_loss did not improve
8000/8000 [==============================] - 76s 10ms/step - loss: 195.7826 - pearson: 0.2913 - mean_squared_error: 195.7826 - mean_absolute_error: 5.2160 - val_loss: 70.3300 - val_pearson: 0.1836 - val_mean_squared_error: 70.3300 - val_mean_absolute_error: 2.5340
Epoch 256/300
7936/8000 [============================>.] - ETA: 0s - loss: 196.893

7936/8000 [============================>.] - ETA: 0s - loss: 193.4994 - pearson: 0.2883 - mean_squared_error: 193.4994 - mean_absolute_error: 5.0793
Epoch 00271: val_loss did not improve
8000/8000 [==============================] - 77s 10ms/step - loss: 195.4747 - pearson: 0.2884 - mean_squared_error: 195.4747 - mean_absolute_error: 5.0907 - val_loss: 70.2566 - val_pearson: 0.1850 - val_mean_squared_error: 70.2566 - val_mean_absolute_error: 2.5516
Epoch 272/300
7936/8000 [============================>.] - ETA: 0s - loss: 195.2041 - pearson: 0.2905 - mean_squared_error: 195.2041 - mean_absolute_error: 5.1452
Epoch 00272: val_loss did not improve
8000/8000 [==============================] - 76s 10ms/step - loss: 195.9179 - pearson: 0.2898 - mean_squared_error: 195.9179 - mean_absolute_error: 5.1498 - val_loss: 70.3000 - val_pearson: 0.1852 - val_mean_squared_error: 70.3000 - val_mean_absolute_error: 2.6474
Epoch 273/300
7936/8000 [============================>.] - ETA: 0s - loss: 195.735

Epoch 289/300
7936/8000 [============================>.] - ETA: 0s - loss: 196.0751 - pearson: 0.2898 - mean_squared_error: 196.0752 - mean_absolute_error: 5.1718
Epoch 00289: val_loss did not improve
8000/8000 [==============================] - 76s 10ms/step - loss: 195.6058 - pearson: 0.2904 - mean_squared_error: 195.6058 - mean_absolute_error: 5.1669 - val_loss: 70.3414 - val_pearson: 0.1827 - val_mean_squared_error: 70.3414 - val_mean_absolute_error: 2.5053
Epoch 290/300
1000/1000 [==============================] - 4s 4ms/steposs: 195.4745 - pearson: 0.291
[('loss', 115.23773043823242), ('pearson', 0.27463336479663847), ('mean_squared_error', 115.23773399353027), ('mean_absolute_error', 3.5915585765838625)]
Test spearman: 0.299776586098

Epoch 00290: val_loss did not improve
8000/8000 [==============================] - 85s 11ms/step - loss: 195.4082 - pearson: 0.2918 - mean_squared_error: 195.4082 - mean_absolute_error: 5.0883 - val_loss: 70.1928 - val_pearson: 0.1846 - val_mean_sq